In [1]:


from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
%matplotlib inline
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
df1=pd.read_excel('/home/ayushi/Downloads/RE/R1.xlsx',sheet_name='Sheet1', header=1)

In [154]:
df1 = df1.iloc[:-1] # deleting last row

df1['Calendar Year Month']=pd.to_datetime(df1['Calendar Year Month'],format='%Y%m')
df1['year']=df1['Calendar Year Month'].apply(lambda x: x.year) 
df1['month']=df1['Calendar Year Month'].apply(lambda x: x.month)
df1.drop(['Calendar Year Month'],axis=1,inplace=True)

In [10]:
df1.groupby('')

1        EuropeMidEastAfrica
2        EuropeMidEastAfrica
3          North East Region
1296     EuropeMidEastAfrica
1297     EuropeMidEastAfrica
2603     EuropeMidEastAfrica
2604     EuropeMidEastAfrica
2605          House Accounts
3903     EuropeMidEastAfrica
3904     EuropeMidEastAfrica
3905          House Accounts
5219     EuropeMidEastAfrica
6560     EuropeMidEastAfrica
6561     EuropeMidEastAfrica
6562          House Accounts
7890     EuropeMidEastAfrica
7891     EuropeMidEastAfrica
9214     EuropeMidEastAfrica
10504    EuropeMidEastAfrica
10505         House Accounts
10506         House Accounts
11809    EuropeMidEastAfrica
11810    EuropeMidEastAfrica
11811         House Accounts
13148    EuropeMidEastAfrica
13149    EuropeMidEastAfrica
13150         House Accounts
14457    EuropeMidEastAfrica
14458    EuropeMidEastAfrica
14459         House Accounts
15708    EuropeMidEastAfrica
15709         House Accounts
15710         House Accounts
16981    EuropeMidEastAfrica
16982    Europ

In [155]:
df1.head()

,Product Line Desc (M),Product Class Desc (M),Sales District Desc (M),ABC indicator,Unnamed: 5,USD,USD.1,USD.2,USD.3,USD.4,USD.5,Unnamed: 12,year,month
0,Arresters,#,North East Region,##,0.00,0.0,0.00,0.0,536.34,0.0,0.0,536.34,2017,1
1,Arresters,Blocks,EuropeMidEastAfrica,B,0.00,0.0,0.00,0.0,23040.00,23040.0,15360.0,61440.00,2017,1
2,Arresters,Blocks,EuropeMidEastAfrica,C,60588.00,60588.0,0.00,0.0,25486.20,46656.0,190256.0,322986.20,2017,1
3,Arresters,Blocks,North East Region,C,0.00,0.0,0.00,0.0,5691.00,0.0,0.0,5691.00,2017,1
4,Arresters,Distribution Arresters,APAC,1,9963.52,0.0,9963.52,0.0,0.00,0.0,0.0,9963.52,2017,1


In [156]:
arr=['Cable Accessories', 'Connectors','Construction', 'Insulators']

In [157]:
def cleaning(x):
    a=df1[df1['Product Line Desc (M)']==x]
    a.drop(['Product Line Desc (M)'],axis=1,inplace=True)
    le = preprocessing.LabelEncoder()
    a['Sales District Desc (M) '] = le.fit_transform(a['Sales District Desc (M) '])
    one_hot1 = pd.get_dummies(a['Product Class Desc (M)'])
    one_hot2 = pd.get_dummies(a['ABC indicator'])
    a = pd.concat([a, one_hot1], axis=1)
    a=pd.concat([a,one_hot2], axis=1)
    a.drop(['Product Class Desc (M)','ABC indicator'],axis=1,inplace=True)
    a.drop(['USD','USD.1','USD.2','USD.3','USD.4','USD.5','Unnamed: 12'],inplace=True,axis=1)
    return a

In [158]:
cable=cleaning('Cable Accessories')
cable.head()

/home/ayushi/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Sales District Desc (M),Unnamed: 5,year,month,URD Accessories,URD Arresters,1,A,B,C,D
181,1,2791.66,2017,1,1,0,0,1,0,0,0
182,1,5512.37,2017,1,1,0,0,0,1,0,0
183,1,0.00,2017,1,1,0,0,0,0,0,1
184,3,0.00,2017,1,1,0,0,1,0,0,0
185,3,0.00,2017,1,1,0,0,0,1,0,0


In [159]:
cable.columns

Index(['Sales District Desc (M) ', 'Unnamed: 5', 'year', 'month',
       'URD Accessories', 'URD Arresters', '1', 'A', 'B', 'C', 'D'],
      dtype='object')

In [160]:
Y=cable['Unnamed: 5']
cable.drop('Unnamed: 5',axis=1,inplace=True)

In [161]:
def xgboost(a):
    X=a
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    model= XGBRegressor(
     learning_rate =0.1,
     n_estimators=1000,
     max_depth=3,
     min_child_weight=5,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'reg:linear',
     nthread=4,
     scale_pos_weight=1,
     seed=27)
    model.fit(X_train,y_train)
    predictions=model.predict(X_test)
    r2=r2_score(y_test,predictions)
    return predictions, r2

In [162]:
cable_xb_pred=xgboost(cable)[0]

In [163]:
cable_xb_r2=xgboost(cable)[1]

In [164]:
cable_xb_r2

0.39044953062492027

In [165]:
lm=linear_model.LinearRegression()
def linear(x):
    X=x
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    model1=lm.fit(X_train, y_train)
    predictions1=lm.predict(X_test)
    r2=r2_score(y_test,predictions1)
    return predictions1,r2

In [166]:
cable_lin_pred=linear(cable)[0]

In [167]:
cable_lin_r2=linear(cable)[1]
cable_lin_r2

0.15615478930272186

In [168]:
def ensemble1(x):
    X=x
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    final_pred=0.8*cable_xb_pred+0.5*cable_lin_pred
    r2=r2_score(y_test,final_pred)
    return final_pred,r2

In [169]:
cable_ensem_pred=ensemble1(cable)[0]

In [170]:
cable_ensem_r2=ensemble1(cable)[1]
cable_ensem_r2

0.40613356791498856

In [171]:
connectors=cleaning('Connectors')

/home/ayushi/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [172]:
Y=connectors['Unnamed: 5']
connectors.drop('Unnamed: 5',axis=1,inplace=True)

In [173]:
connectors.head()

,Sales District Desc (M),year,month,Aluminium Loops,Distribution Connectors,Plastic Loops,Substation Connectors,Transmission Connectors,##,1,A,B,C,D,P
239,3,2017,1,1,0,0,0,0,0,1,0,0,0,0,0
240,3,2017,1,1,0,0,0,0,0,0,1,0,0,0,0
241,3,2017,1,1,0,0,0,0,0,0,0,1,0,0,0
242,3,2017,1,1,0,0,0,0,0,0,0,0,1,0,0
243,4,2017,1,1,0,0,0,0,0,1,0,0,0,0,0


In [174]:
connectors_xg_pred=xgboost(connectors)[0]

In [175]:
connectors_xg_r2=xgboost(connectors)[1]
connectors_xg_r2

0.49167487398426757

In [176]:
connectors_lin_pred=linear(connectors)[0]

In [177]:
connectors_lin_r2=linear(connectors)[1]
connectors_lin_r2

0.2386250319640103

In [178]:
connectors_xg_pred

array([ 2322.0408,  7841.2827, 20270.98  , ..., -6644.2197,  4190.03  ,
       -3639.9277], dtype=float32)

In [179]:
def ensemble2(x):
    X=x
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    final_pred=0.8*connectors_xg_pred+0.5*connectors_lin_pred
    r2=r2_score(y_test,final_pred)
    return final_pred,r2

In [180]:
connectors_ensem_pred=ensemble2(connectors)[0]

In [181]:
connectors_ensem_r2=ensemble2(connectors)[1]
connectors_ensem_r2

0.4332794412305485

In [182]:
construction=cleaning('Construction')

/home/ayushi/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [183]:
construction.head()

,Sales District Desc (M),Unnamed: 5,year,month,#,Atlas,Civil,Pole Line Hardware,Utility Anchors,##,1,A,B,C,D,P
404,10,1500.0,2017,1,1,0,0,0,0,1,0,0,0,0,0,0
405,12,1000.0,2017,1,1,0,0,0,0,1,0,0,0,0,0,0
406,2,1436.0,2017,1,0,1,0,0,0,0,0,1,0,0,0,0
407,2,27525.8,2017,1,0,1,0,0,0,0,0,0,1,0,0,0
408,2,807.0,2017,1,0,1,0,0,0,0,0,0,0,1,0,0


In [184]:
Y=construction['Unnamed: 5']
construction.drop('Unnamed: 5',axis=1,inplace=True)

In [185]:
construction_xg_pred=xgboost(construction)[0]

In [186]:
construction_xg_r2=xgboost(construction)[1]
construction_xg_r2

0.6871317301236557

In [187]:
construction_lin_pred=linear(construction)[0]

In [188]:
construction_lin_r2=linear(construction)[1]
construction_lin_r2

0.15592786630261846

In [189]:
def ensemble3(x):
    X=x
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    final_pred=0.8*construction_xg_pred+0.5*construction_lin_pred
    r2=r2_score(y_test,final_pred)
    return final_pred,r2

In [190]:
construction_ensem_pred=ensemble3(construction)[0]

In [191]:
construction_ensem_r2=ensemble3(construction)[1]
construction_ensem_r2

0.6478699837671893

In [192]:
insulators=cleaning('Insulators')

/home/ayushi/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [193]:
insulators.head()

,Sales District Desc (M),Unnamed: 5,year,month,#,Distribution Insulators,Transmission Insulators,##,1,A,B,C,D
645,0,1792.50,2017,1,0,1,0,0,1,0,0,0,0
646,0,0.00,2017,1,0,1,0,0,0,1,0,0,0
647,0,9384.00,2017,1,0,1,0,0,0,0,1,0,0
648,0,84.00,2017,1,0,1,0,0,0,0,0,1,0
649,1,6891.17,2017,1,0,1,0,0,0,1,0,0,0


In [194]:
Y=insulators['Unnamed: 5']
insulators.drop('Unnamed: 5',axis=1,inplace=True)

In [195]:
insulators_xg_pred=xgboost(insulators)[0]

In [196]:
insulators_xg_r2=xgboost(insulators)[1]
insulators_xg_r2

0.1031413484652145

In [197]:
insulators_lin_pred=linear(insulators)[0]

In [198]:
insulators_lin_r2=linear(insulators)[1]
insulators_lin_r2

0.083124435157796

In [199]:
def ensemble4(x):
    X=x
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    final_pred=0.8*insulators_xg_pred+0.5*insulators_lin_pred
    r2=r2_score(y_test,final_pred)
    return final_pred,r2

In [200]:
insulators_ensem_pred=ensemble4(insulators)[0]

In [201]:
insulators_ensem_r2=ensemble4(insulators)[1]
insulators_ensem_r2

0.1227164994607044

In [205]:
df2=df1[df1['Product Line Desc (M)'].isin(['Cable Accessories', 'Connectors','Construction', 'Insulators'])]

In [206]:
le = preprocessing.LabelEncoder()
df2['Sales District Desc (M) '] = le.fit_transform(df2['Sales District Desc (M) '])
df2['Product Line Desc (M)']=le.fit_transform(df2['Product Line Desc (M)'])
one_hot1 = pd.get_dummies(df2['Product Class Desc (M)'])
one_hot2 = pd.get_dummies(df2['ABC indicator'])
df2 = pd.concat([df2, one_hot1], axis=1)
df2=pd.concat([df2,one_hot2], axis=1)
df2.drop(['Product Class Desc (M)','ABC indicator'],axis=1,inplace=True)
df2.drop(['USD','USD.1','USD.2','USD.3','USD.4','USD.5','Unnamed: 12'],inplace=True,axis=1)

/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [210]:
df2.drop(['#','##'],axis=1,inplace=True)

In [211]:
df2.head()

,Product Line Desc (M),Sales District Desc (M),Unnamed: 5,year,month,Aluminium Loops,Atlas,Civil,Distribution Connectors,Distribution Insulators,...,Transmission Insulators,URD Accessories,URD Arresters,Utility Anchors,1,A,B,C,D,P
181,0,1,2791.66,2017,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
182,0,1,5512.37,2017,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
183,0,1,0.00,2017,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
184,0,4,0.00,2017,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
185,0,4,0.00,2017,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
